### Method 1 - Disturb, fetch in order, loop when attain the end.

In [1]:
import numpy as np 

class Dataset:

    def __init__(self,data):
        self._index_in_epoch = 0
        self._epochs_completed = 0
        self._data = data
        self._num_examples = data.shape[0]
        pass

    @property
    def data(self):
        return self._data

    def next_batch(self,batch_size,shuffle = True):
        start = self._index_in_epoch
        if start == 0 and self._epochs_completed == 0:
            idx = np.arange(0, self._num_examples)  # get all possible indexes
            np.random.shuffle(idx)  # shuffle indexe
            self._data = self.data[idx]  # get list of `num` random samples

        # go to the next batch
        if start + batch_size > self._num_examples:
            self._epochs_completed += 1
            rest_num_examples = self._num_examples - start
            data_rest_part = self.data[start:self._num_examples]
            idx0 = np.arange(0, self._num_examples)  # get all possible indexes
            np.random.shuffle(idx0)  # shuffle indexes
            self._data = self.data[idx0]  # get list of `num` random samples

            start = 0
            self._index_in_epoch = batch_size - rest_num_examples #avoid the case where the #sample != integar times of batch_size
            end =  self._index_in_epoch  
            data_new_part =  self._data[start:end]  
            return np.concatenate((data_rest_part, data_new_part), axis=0)
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            return self._data[start:end]

In [2]:
num = 7
features = np.random.normal(0,1,(num,3))
labels = np.random.normal(0,1,(num,2))
train_data = np.hstack((features, labels))
print(train_data)

[[ 0.044405   -1.80456467 -1.06199452 -0.14901493 -0.17632282]
 [-0.14319513 -1.07524987  0.43672162 -0.37384503  0.66633331]
 [ 0.78068873  0.25609399 -0.05201155  1.74364849  1.7539313 ]
 [-1.28008302 -0.72534653  0.46229277  0.20281318 -0.66613969]
 [ 0.73371191  0.14944569  1.29800851  0.58278309  0.27298921]
 [ 1.11325289 -0.76692396 -0.74869919  0.83903653 -0.33771925]
 [-0.33432592 -0.27145528  4.07791904  2.12370519 -1.14545456]]


In [3]:
batch_size = 3
dataset = Dataset(train_data)
for i in range(10):
    data_batch = dataset.next_batch(batch_size)
    batch_features, batch_labels = data_batch[:, :len(features[0])], data_batch[:, len(features[0]):]
    print(i, batch_features, batch_labels)

(0, array([[-1.28008302, -0.72534653,  0.46229277],
       [-0.14319513, -1.07524987,  0.43672162],
       [ 0.044405  , -1.80456467, -1.06199452]]), array([[ 0.20281318, -0.66613969],
       [-0.37384503,  0.66633331],
       [-0.14901493, -0.17632282]]))
(1, array([[ 0.78068873,  0.25609399, -0.05201155],
       [-0.33432592, -0.27145528,  4.07791904],
       [ 0.73371191,  0.14944569,  1.29800851]]), array([[ 1.74364849,  1.7539313 ],
       [ 2.12370519, -1.14545456],
       [ 0.58278309,  0.27298921]]))
(2, array([[ 1.11325289, -0.76692396, -0.74869919],
       [ 0.73371191,  0.14944569,  1.29800851],
       [ 0.78068873,  0.25609399, -0.05201155]]), array([[ 0.83903653, -0.33771925],
       [ 0.58278309,  0.27298921],
       [ 1.74364849,  1.7539313 ]]))
(3, array([[-1.28008302, -0.72534653,  0.46229277],
       [ 1.11325289, -0.76692396, -0.74869919],
       [-0.14319513, -1.07524987,  0.43672162]]), array([[ 0.20281318, -0.66613969],
       [ 0.83903653, -0.33771925],
       [-

### Method 2 Disturb and fetch every time

In [1]:
def next_batch(x, y, batch_size):
    N = x.shape[0]
    batch_indices = np.random.permutation(N)[:batch_size]
    return x[batch_indices], y[batch_indices]